In [ ]:
import os
import pandas as pd 
import matplotlib.pyplot as plt 
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np


from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
labels = pd.read_csv('../input/clothing-dataset-full/images.csv')
labels.head()

labels['image'] = labels['image'] + '.jpg'
label_df = labels[['image', 'label']]
classes=list(label_df['label'].unique())
repl={}
for i in range(len(classes)):
    repl[classes[i]]=i
for i in range(len(label_df)):
    label_df['label'][i]=repl[label_df['label'][i]]

from sklearn.model_selection import train_test_split
train_label_df, test_label_df = train_test_split(label_df, test_size=0.10)
train_label_df.to_csv ('./train_csv.csv', index = False, header=True)
test_label_df.to_csv ('./test_csv.csv', index = False, header=True)
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class DressDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = Image.open(self.root_dir+ img_id).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)
transform = transforms.Compose(
    [transforms.Resize((1000,1000)),transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 1

trainset = DressDataset(root_dir='../input/clothing-dataset-full/images_original/', annotation_file='./train_csv.csv', transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = DressDataset(root_dir='../input/clothing-dataset-full/images_original/', annotation_file='./test_csv.csv', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)


In [ ]:
class Conv1(torch.nn.Module):
    def __init__(self):
        
        super().__init__()
        self.weight1 = nn.Parameter(torch.randn(3, 20, 20))
        self.weight2 = nn.Parameter(torch.randn(3, 20, 20))

    def forward(self, x):
        
        batch_size = 1
        channels = 3
        image = x # input image

        kh, kw = 20, 20 # kernel size
        dh, dw = 20, 20 # stride

        # Create conv
        
        inputs=x
        patches = inputs.unfold(2, kh, dh).unfold(3, kw, dw)
        #print(patches.shape)
        patches = patches.contiguous().view(batch_size, channels, -1, kh, kw)
        #print(patches.shape)
        nb_windows = patches.size(2)

       
        patches = patches.permute(0, 2, 1, 3, 4)
        #print(patches.shape)
        patches = patches.view(-1, channels, kh, kw)
        #print(patches.shape)

        conv = nn.Conv2d(channels, batch_size, (kh, kw), stride=(dh, dw), bias=False)
        patches = patches * torch.mul(self.weight1, self.weight2)
        patches = patches.sum(1)
        patches=patches.unsqueeze(0)
        return patches

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =Conv1()
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(250000, 20)
        

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
losses=[]
for epoch in range(6):  

    running_loss = 0.0
    i=0
    for inputs, labels in trainloader:
        
        optimizer.zero_grad()
        outputs = net(inputs.float())
        loss = criterion(outputs,labels.long())
        loss.backward()
        optimizer.step()        
        running_loss += loss.item()
        i+=1
        if i % 200 == 0:    
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    with torch.no_grad():
        val_loss=0
        for val_inputs,val_labels in testloader:
            val_outputs = net(val_inputs.float())
            loss = criterion(val_outputs,val_labels.long())     
            val_loss += loss.item()
        losses.append(val_loss)
        print("Validation loss:",val_loss)

print('Finished Training')

In [ ]:
plt.plot(losses)

In [ ]:
torch.save(net, "./demo1.pth")